In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from functions.load_data import *
from functions.plotting import *
from tqdm import tqdm
import pickle

In [ ]:
ds_all, ds_all_centered, datasets, datasets_centered, ds_counts = load_data(pattern="../data/mariel_*.npy")

In [ ]:
ds_all_centered.shape

In [ ]:
ds_all_centered[:5,0,0]

In [ ]:
seq_len = 10
index_start = 0
# index_start = np.random.randint(0,len(ds_all_centered)-seq_len)
print("Seeding with frame {}".format(index_start))
xtest = ds_all_centered[index_start:index_start+seq_len,:,:3]
animation = animate_stick(xtest, 
                          figsize=(10,8), 
                          cmap='inferno', 
                          cloud=False
                         )
HTML(animation.to_html5_video())

# Convert to NRI format

In [ ]:
print(ds_all_centered.shape)
ds_all_centered = np.transpose(ds_all_centered, [0, 2, 1])
print(ds_all_centered.shape)

### Optional: create reduced-joints training set (31 joints, to match NRI)

In [ ]:
# point_labels=['ARIEL','C7','CLAV','LANK','LBHD','LBSH','LBWT','LELB','LFHD',
#           'LFRM','LFSH','LFWT','LHEL','LIEL','LIHAND','LIWR','LKNE','LKNI',
#           'LMT1','LMT5','LOHAND','LOWR','LSHN','LTHI','LTOE','LUPA','MBWT',
#           'MFWT','RANK','RBHD','RBSH','RBWT','RELB','RFHD','RFRM','RFSH',
#           'RFWT','RHEL','RIEL','RIHAND','RIWR','RKNE','RKNI','RMT1','RMT5',
#           'ROHAND','ROWR','RSHN','RTHI','RTOE','RUPA','STRN','T10']

# ### Define a subset of joints if we want to train on fewer joints that still capture meaningful body movement:
# reduced_joint_names = ['ARIEL', 'RFHD', 'LFHD', 'RBHD', 'LBHD', 'CLAV', 'C7', 'RFSH', 'LFSH', 'RIEL', 'LIEL', 'RIWR', 'LIWR','RKNE','LKNE','RTOE','LTOE','LHEL','RHEL','LANK','RANK','RKNI','LKNI','ROWR','LOWR','RELB','LELB','RFWT','LFWT','LBWT','RBWT']
# reduced_joint_indices = [point_labels.index(joint_name) for joint_name in reduced_joint_names]
# print(len(reduced_joint_indices))

In [ ]:
# data = ds_all_centered[:,:,reduced_joint_indices]
# print(data.shape)

### Start with creating the train & validation sets...

In [ ]:
data = ds_all_centered

In [ ]:
n_simulations = 500
seq_length = 49
n_joints = data.shape[2]
nodes = []
edges = []

start = 0
for n in range(n_simulations):
    sequence = data[start:start+seq_length]
    nodes.append(sequence)
    edges.append(np.ones((n_joints,n_joints))) # fully-connected
    start += seq_length+1

nodes = np.stack(nodes)
edges = np.stack(edges)

In [ ]:
print(nodes.shape)
print(edges.shape)

In [ ]:
nodes[0,:5,0,0]

In [ ]:
sequence = nodes[0]
print(sequence.shape)
sequence = np.transpose(sequence, [0,2,1])

animation = animate_stick(sequence[:,:,:3], figsize=(10,8), cmap='inferno', cloud=False)
HTML(animation.to_html5_video())

In [ ]:
# def unison_shuffled_copies(a, b, c):
#     assert len(a) == len(b) == len(c)
#     p = np.random.permutation(len(a))
#     return a[p], b[p], c[p]

# nodes, velocities, edges = unison_shuffled_copies(nodes, velocities, edges) # shuffle all of them in the same way

In [ ]:
train_index = int(n_simulations*(.8)) # first 80% go to the training set; remaining 20% go to the validation set
print(train_index)

In [ ]:
suffix = "_dance_noshuffle"
np.save("../../NRI/data/loc_train"+suffix+".npy", nodes[:train_index, :, :3, :])
np.save("../../NRI/data/vel_train"+suffix+".npy", nodes[:train_index, :, 3:, :])
np.save("../../NRI/data/edges_train"+suffix+".npy", edges[:train_index])

np.save("../../NRI/data/loc_valid"+suffix+".npy", nodes[train_index:, :, :3, :])
np.save("../../NRI/data/vel_valid"+suffix+".npy", nodes[train_index:, :, 3:, :])
np.save("../../NRI/data/edges_valid"+suffix+".npy", edges[train_index:])

Preview the samples you just saved:

In [ ]:
suffix = "_dance_noshuffle"

In [ ]:
nodes = np.load("../../NRI/data/loc_train"+suffix+".npy")
velocities = np.load("../../NRI/data/vel_train"+suffix+".npy")
edges = np.load("../../NRI/data/edges_train"+suffix+".npy")
i = 0
seq_len = 49

sequence = nodes[i]
sequence = np.transpose(sequence, [0,2,1])

# only do this if not using reduced joints:
animation = animate_stick(sequence[:,:,:3], figsize=(10,8), cmap='inferno', cloud=False)
HTML(animation.to_html5_video())

In [ ]:
nodes.shape

In [ ]:
# first 10 timesteps of joint 0_x: 
nodes[0,:5,0,0]

### Now make the test set, with a longer sequence length (1000 timesteps), starting after timestep 49*500 = 24,500

In [ ]:
(38309-24500)/1000

In [ ]:
n_simulations = 13
seq_length = 1000
nodes = []
edges = []
start = 24500

for n in range(n_simulations):
    sequence = data[start:start+seq_length]
    nodes.append(sequence)
    edges.append(np.ones((n_joints,n_joints))) # fully-connected
    start += seq_length+1

nodes = np.stack(nodes)
edges = np.stack(edges)

In [ ]:
print(nodes.shape)
print(edges.shape)

In [ ]:
suffix = "_dance_noshuffle"

In [ ]:
np.save("../../NRI/data/loc_test"+suffix+".npy", nodes[:, :, :3, :])
np.save("../../NRI/data/vel_test"+suffix+".npy", nodes[:, :, 3:, :])
np.save("../../NRI/data/edges_test"+suffix+".npy", edges)

Preview the test set you just saved:

In [ ]:
suffix = "_dance_noshuffle"
n_joints = 53
seq_len = 1000

In [ ]:
nodes = np.load("../../NRI/data/loc_test"+suffix+".npy") 
velocities = np.load("../../NRI/data/vel_test"+suffix+".npy")
edges = np.load("../../NRI/data/edges_test"+suffix+".npy")

In [ ]:
i = 0
seq_len = 1000
sequence = nodes[i]
sequence = np.transpose(sequence, [0,2,1])
animation = animate_stick(sequence[:,:,:3], figsize=(10,8), cmap='inferno', cloud=False)
HTML(animation.to_html5_video())